In [1]:
import torch
from torch import nn
import torch.nn.functional as F
from utils import View
from utils import train_network
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets
from torchvision.datasets import MNIST
from torchvision import transforms
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

d:\projects\git-hub\simple-nonlinear-PCA\utils.py:7: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
D=28*28               # Dimension of the input data
n=2                   # no of hidden layers
C=1                   # no of channels
classes=10            # output classes

In [3]:
class AutoEncodeDataset(Dataset):
    def __init__(self,dataset):
        self.dataset = dataset

    def __len__(self):
        return len(self.dataset)
    
    def __getitem__(self,idx):
        x,y = self.dataset.__getitem__(idx)
        return x , x 

In [4]:
train_data = AutoEncodeDataset(MNIST("./", train=True, transform=transforms.ToTensor(), download=True))
test_data_xy = MNIST("./", train=False, transform=transforms.ToTensor(), download=True)
test_data_xx = AutoEncodeDataset(test_data_xy)

train_loader = DataLoader(train_data, batch_size=128, shuffle=True)
test_loader = DataLoader(test_data_xx, batch_size=128)

100%|██████████| 8674/8674 [00:00<00:00, 8448999.74it/s]


RuntimeError: File not found or corrupted.

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [ ]:
'''
intiate the sequence with flattening the 28*28 img then passing that into linear
with size 28*28 = 784 and output size n = 2
'''
PCA_non_linear_encoder = nn.Sequential(
    nn.Flatten(),
    nn.Linear(D,n),
    nn.Tanh(),
)
'''
intiate the sequence with taking the features of n = 2 thatw ended with into
784 afain using a Linear layer (n,D) then changing the view of these using a internal 
utils function like View or using UnFlatten from torch.nn
'''

PCA_non_linear_decoder = nn.Sequential(
    nn.Linear(n,D),
    View(-1,1,28,28),
)

'''
making the encoder decoder sequential model
'''
PCA_non_linear = nn.Sequential(
    PCA_non_linear_encoder,
    PCA_non_linear_decoder
)

In [ ]:
train_network(PCA_non_linear,loss_func=nn.MSELoss(),)